In [1]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!ls ../

build_run.sh	  image-dehazing.py  run_opsfm.sh
config.yaml	  images	     run.sh
data.bag	  imgs_bak	     set_env.sh
demo		  jupyter_notebooks  test.bag
demonstration.sh  mve_run.sh	     test.ipynb
dev.ipynb	  preproc_ros.py     underwater_image_dehazing
Dockerfile	  requirements.txt   Underwater-Image-Enhancement
enhanced_images   run_demo.sh


In [3]:
import numpy as np
import cv2



PROJ_DIR="/home/jcc2000/Desktop/datahub-docker"
IMG_DIR=f"{PROJ_DIR}/images"
IMG_ENHANCED_DIR=f"{PROJ_DIR}/enhanced_images"


In [4]:
def resize(img, perc=0.5):
    """Utility function to resize the image"""
    width = int(img.shape[1] * perc)
    height = int(img.shape[0] * perc)
    resized = cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)
    return resized


def dark_channel(img, sz=15):
    """Calculates the dark channel in step 1"""
    b, g, r = cv2.split(img)
    dc = r - cv2.max(b, g)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (sz, sz))
    dark = cv2.erode(dc, kernel)
    return dark


def airlight(img, dc):
    """Calculates the air-light in step 4"""
    [h, w] = img.shape[:2]
    imgsz = h * w
    imgvec = img.reshape(imgsz, 3)
    dcvec = dc.reshape(imgsz)

    indices = dcvec.argsort()
    return imgvec[indices[0]]


def transmission_estimate(dc):
    """Calculates the transmission map as mentioned in step 2"""
    te = dc + (1 - np.max(dc))
    return te


def guided_filter(img, p, r, eps):
    """Helper function to refine the transmission"""
    mean_I = cv2.boxFilter(img, cv2.CV_64F, (r, r))
    mean_p = cv2.boxFilter(p, cv2.CV_64F, (r, r))
    mean_Ip = cv2.boxFilter(img * p, cv2.CV_64F, (r, r))
    cov_Ip = mean_Ip - mean_I * mean_p

    mean_II = cv2.boxFilter(img * img, cv2.CV_64F, (r, r))
    var_I = mean_II - mean_I * mean_I

    a = cov_Ip / (var_I + eps)
    b = mean_p - a * mean_I

    mean_a = cv2.boxFilter(a, cv2.CV_64F, (r, r))
    mean_b = cv2.boxFilter(b, cv2.CV_64F, (r, r))

    q = mean_a * img + mean_b
    return q


def transmission_refine(img, et):
    """Refines the transmission map as mentioned in step 3"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray) / 255
    r = 60
    eps = 0.0001
    t = guided_filter(gray, et, r, eps)
    return t


def recover(img, t, A):
    """Finally recovers the image using the image, the transmission map and air-light"""
    res = np.empty(img.shape, img.dtype)
    for index in range(0, 3):
        res[:, :, index] = (img[:, :, index] - A[index]) / t + A[index]

    return res


def normalize_image(img):
    """Utility function to normalize the intensities of the pixels in the image"""
    img = img - img.min()
    img = img / img.max() * 255
    img = np.uint8(img)
    return img


def dummy(x):
    pass


In [5]:

"""
if __name__ == '__main__':
    filename = 'color_img1663755872636549118.jpg'
    scale = 1

    src = cv2.imread(f'{IMG_DIR}/{filename}')
    src = resize(src, scale)

    img = src.astype('float64') / 255

#     cv2.namedWindow('controls')
#     cv2.createTrackbar('ksize', 'controls', 1, 600, dummy)

#     while True:
# #         ksize = int(cv2.getTrackbarPos('ksize', 'controls')) + 1
#         ksize = int(128)

#         dc = dark_channel(img, ksize)
#         te = transmission_estimate(dc)
#         tr = transmission_refine(src, te)
#         A = airlight(img, tr)
#         result = recover(img, tr, A)
#         result = normalize_image(result)

#         cv2.imshow('Transmission estimate', te)
#         cv2.imshow('Transmission refined', tr)
#         cv2.imshow('Input', img)
#         cv2.imshow('Result', result)

#         k = cv2.waitKey(1) & 0xFF
#         if k == 27:
#             break
    ksize = int(128)

    dc = dark_channel(img, ksize)
    te = transmission_estimate(dc)
    tr = transmission_refine(src, te)
    A = airlight(img, tr)
    result = recover(img, tr, A)
    result = normalize_image(result)

    cv2.imwrite(f'{IMG_ENHANCED_DIR}/{filename}', result)
    
"""

"\nif __name__ == '__main__':\n    filename = 'color_img1663755872636549118.jpg'\n    scale = 1\n\n    src = cv2.imread(f'{IMG_DIR}/{filename}')\n    src = resize(src, scale)\n\n    img = src.astype('float64') / 255\n\n#     cv2.namedWindow('controls')\n#     cv2.createTrackbar('ksize', 'controls', 1, 600, dummy)\n\n#     while True:\n# #         ksize = int(cv2.getTrackbarPos('ksize', 'controls')) + 1\n#         ksize = int(128)\n\n#         dc = dark_channel(img, ksize)\n#         te = transmission_estimate(dc)\n#         tr = transmission_refine(src, te)\n#         A = airlight(img, tr)\n#         result = recover(img, tr, A)\n#         result = normalize_image(result)\n\n#         cv2.imshow('Transmission estimate', te)\n#         cv2.imshow('Transmission refined', tr)\n#         cv2.imshow('Input', img)\n#         cv2.imshow('Result', result)\n\n#         k = cv2.waitKey(1) & 0xFF\n#         if k == 27:\n#             break\n    ksize = int(128)\n\n    dc = dark_channel(img, ksize

In [6]:
from os import listdir
from os.path import isfile, join

print(IMG_DIR)

onlyfiles = [IMG_DIR+"/"+f for f in listdir(IMG_DIR) if isfile(join(IMG_DIR, f))]
onlyimgs = [f for f in listdir(IMG_DIR) if isfile(join(IMG_DIR, f))]

/home/jcc2000/Desktop/datahub-docker/images


In [7]:
onlyimgs[0]

'1663755905469884128.jpg'

In [8]:

scale = 1
for f in onlyfiles:

    
    src = cv2.imread(f)
    src = resize(src, scale)

    img = src.astype('float64') / 255

    ksize = int(128)

    dc = dark_channel(img, ksize)
    te = transmission_estimate(dc)
    tr = transmission_refine(src, te)
    A = airlight(img, tr)
    result = recover(img, tr, A)
    result = normalize_image(result)
    
    filename = f.split("/")[-1]


    cv2.imwrite(f'{IMG_ENHANCED_DIR}/{filename}', result)

    print(f'{IMG_ENHANCED_DIR}/{filename}')



/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755905469884128.jpg
/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755864362543193.jpg
/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755888139362282.jpg
/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755814920513402.jpg
/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755860321248531.jpg
/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755803303880499.jpg
/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755931401470511.jpg
/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755904948011421.jpg
/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755818821790315.jpg
/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755856375435223.jpg
/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755868377162855.jpg
/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755911964667534.jpg
/home/jcc2000/Desktop/datahub-docker/enhanced_images/1663755818009722680.jpg

In [9]:
onlyfiles

['/home/jcc2000/Desktop/datahub-docker/images/1663755905469884128.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/1663755864362543193.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/1663755888139362282.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/1663755814920513402.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/1663755860321248531.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/1663755803303880499.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/1663755931401470511.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/1663755904948011421.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/1663755818821790315.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/1663755856375435223.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/1663755868377162855.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/1663755911964667534.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/1663755818009722680.jpg',
 '/home/jcc2000/Desktop/datahub-docker/images/166375589330587262

In [10]:
!pip3 install image_dehazer

Defaulting to user installation because normal site-packages is not writeable


In [11]:
import image_dehazer # Load the library
import cv2

In [12]:
HazeImg = cv2.imread(f'{IMG_DIR}/color_img1663755872636549118.jpg')# read input image -- (**must be a color image**)
HazeCorrectedImg = image_dehazer.remove_haze(HazeImg, showHazeTransmissionMap=False)# Remove Haze


[ WARN:0@23.793] global loadsave.cpp:244 findDecoder imread_('/home/jcc2000/Desktop/datahub-docker/images/color_img1663755872636549118.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
cv2.imwrite(f'{IMG_ENHANCED_DIR}/color_img1663755872636549118_1.jpg', HazeCorrectedImg[0])
# cv2.imshow('input image', HazeImg);# display the original hazy image
# cv2.imshow('enhanced_image', HazeCorrectedImg[1]);# display the result
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
cv2.imwrite(f'{IMG_ENHANCED_DIR}/color_img1663755872636549118_2.jpg', HazeCorrectedImg[1])

In [ ]:
cv2.imshow('input image', HazeImg);# display the original hazy image
cv2.imshow('enhanced_image', HazeCorrectedImg);# display the result
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite(f'{IMG_ENHANCED_DIR}/color_img1663755888903749289.jpg', HazeCorrectedImg)